# Exploring the Point Generators

In this notebook we explore the three different point generator classes.

1. The *PointGenerator* class is the base class the other two inherit from. It can be used for Complete Intersection Calabi-Yau (CICY) manifolds with a single hypersurface, e.g. the quintic, bicubic, p1p3, the tetraquadric, or a torus, some K3s ...
2. There is the *CICYPointGenerator* which works for any CICY.
3. There is the *PointGeneratorMathematica* which works for any CICY (also for just a single hypersurface. It requires a Mathematica installation which will be used as a backend for the computations.
4. There is the *ToricPointGenerator* which can generate points on any CY coming from the Kreuzer-Skarke list. NOTE: as of v0.0.1 there are still some issues with finding the correct integration weights from generalizing a theorem by Shiffman and Zelditch. For the correct implementation use *ToricPointGeneratorMathematica*.

The Fermat quintic manifold can be implemented in each of the three PointGenerator classes and will be used as an example.

Recall that the Fermat quintic is given by the following polynomial:

$$
Q = \sum_i^5 z_i^5
$$

in a $\mathbb{P}^4$ ambient space.

In [1]:
import numpy as np
import os as os
import pickle as pickle

## PointGenerator

First we load the PointGenerator from the cymetric package.

In [2]:
from cymetric.pointgen.pointgen import PointGenerator

In general all routines and classes of the cymetric package will have a brief description of their functionality and their arguments. You can access it with the help function.

In [3]:
help(PointGenerator)

Help on class PointGenerator in module cymetric.pointgen.pointgen:

class PointGenerator(builtins.object)
 |  PointGenerator(monomials, coefficients, kmoduli, ambient, vol_j_norm=None, verbose=2, backend='multiprocessing')
 |  
 |  The PointGenerator class.
 |  
 |  The numerics are entirely done in numpy; sympy is used for taking 
 |  (implicit) derivatives.
 |  
 |  Use this one if you want to generate points and data on a CY given by
 |  one hypersurface.
 |  
 |  All other PointGenerators inherit from this class.
 |  
 |  Example:
 |      We consider the Fermat quintic given by
 |  
 |      .. math::
 |  
 |          Q(z) = z_1^5 + z_2^5 + z_3^5 + z_4^5 + z_5^5
 |  
 |      and set it up with:
 |  
 |      >>> import numpy as np
 |      >>> from cymetric.pointgen.pointgen import PointGenerator
 |      >>> monomials = 5*np.eye(5, dtype=np.int)
 |      >>> coefficients = np.ones(5)
 |      >>> kmoduli = np.ones(1)
 |      >>> ambient = np.array([4])
 |      >>> pg = PointGenerator(mo

In the next step we define the five defining monomials, the coefficients in front of each monomial, the single Kähler moduli, and the ambient space projective factor(s):

In [3]:
monomials = 5*np.eye(5, dtype=np.int64)
coefficients = np.ones(5)
kmoduli = np.ones(1)
ambient = np.array([4])

We can now initiate the PointGenerator

In [5]:
pg = PointGenerator(monomials, coefficients, kmoduli, ambient)

and generate some (100) points:

In [6]:
points = pg.generate_points(100)
points

array([[-0.26571881-9.17297455e-01j, -0.36345359+2.42049070e-01j,
         1.        +0.00000000e+00j, -0.59842131-1.43934436e-01j,
        -0.35605605-6.91997143e-01j],
       [ 1.        -5.55111512e-17j, -0.31804611-5.70832600e-01j,
         0.78152219+6.00384439e-01j,  0.15162194-1.69715014e-02j,
         0.31991803-3.70442042e-01j],
       [ 1.        +2.77555756e-17j, -0.23516908-2.60266458e-01j,
        -0.77837294+1.05176966e-01j,  0.81948808-4.76754593e-01j,
         0.66788379+2.93563540e-01j],
       [ 0.8244501 +5.64605799e-01j, -0.16831985-6.66244130e-01j,
         1.        +0.00000000e+00j,  0.28974831+6.27956749e-01j,
        -0.05495057-4.40566702e-02j],
       [-0.18569486+7.50333720e-01j,  0.24012511-3.46593059e-01j,
         1.        +0.00000000e+00j, -0.31459026+8.85538129e-01j,
        -0.45377699-5.09986505e-02j],
       [-0.62309363+5.81626089e-02j, -0.4679331 +6.79700462e-02j,
        -0.54795213-2.22345813e-02j,  1.        +0.00000000e+00j,
        -0.9677924

We see that the largest coordinate of each point is 1+0.j. The reason for that is two-fold:

1. The numerics are more stable when we work with points, which have affine coordinates in the range of $|x_i| < 1$.
2. The coordinate with 1+0.j already specifies the ambient space patch we are working in.

We can also check if these points are all really satisfying the hypersurface equation of the Calabi-Yau.

In [7]:
pg.cy_condition(points)

array([ 1.72084569e-15+1.11022302e-15j, -1.96370697e-15+7.50094431e-15j,
       -1.92901251e-15-6.13398221e-15j, -4.45792318e-15-5.78768899e-15j,
       -3.04270498e-15-2.25514052e-16j,  8.88178420e-16+3.74006381e-15j,
       -1.55431223e-15-4.71844785e-16j, -9.99200722e-16+5.55111512e-17j,
        2.16493490e-15+1.94289029e-15j, -7.38124839e-16+2.63677968e-16j,
        3.02904403e-15-1.26027661e-15j, -2.03682286e-15-3.62199420e-15j,
        1.15879528e-15-3.94649591e-16j,  4.14016152e-16-9.76097051e-16j,
       -2.56088553e-16-3.58220398e-16j, -5.55111512e-17-2.22044605e-16j,
       -2.88657986e-15+4.44089210e-16j,  9.99200722e-16+1.31838984e-15j,
       -6.55031585e-15+2.77555756e-15j,  1.11022302e-15+2.22044605e-16j,
        2.77555756e-15-3.19189120e-16j, -3.77475828e-15-2.27595720e-15j,
       -3.31408078e-15+2.33320308e-15j,  1.52655666e-15+1.55431223e-15j,
       -4.30211422e-16-4.85722573e-16j, -2.30284541e-16-1.68268177e-16j,
        2.67841305e-15-6.93889390e-15j,  2.77555756

what we are really interested in from the *PointGenerator* is a training set for our neural networks. Such a training set can be generated as follows:

In [8]:
help(pg.prepare_dataset)

Help on method prepare_dataset in module cymetric.pointgen.pointgen:

prepare_dataset(n_p, dirname, val_split=0.1, ltails=0, rtails=0) method of cymetric.pointgen.pointgen.PointGenerator instance
    Prepares training and validation data.
    
    Args:
        n_p (int): Number of points to generate.
        dirname (str): Directory name to save dataset in.
        val_split (float, optional): train-val split. Defaults to 0.1.
        ltails (float, optional): Percentage discarded on the left tail
            of weight distribution. Defaults to 0.
        rtails (float, optional): Percentage discarded on the right tail
            of weight distribution. Defaults to 0.
    
    Returns:
        np.float: kappa = vol_k / vol_cy



We specify the number of points and the directory name to save the file in. Note the file will always have the name *dataset.npz*.

In [9]:
dirname = 'fermat_pg'
n_p = 100000

and generate the dataset. This will also compute and return $\kappa=\text{vol}_\text{K}/\text{vol}_\text{CY}$

In [10]:
kappa = pg.prepare_dataset(n_p, dirname);
kappa

/Users/ruehle/venv-ml/lib/python3.8/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/ruehle/venv-ml/lib/python3.8/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
pointgen:INFO:Vol_k: 5.000000000000001, Vol_cy: 284.2561104432773.


0.017589771393842175

We load the dataset with

In [11]:
data = np.load(os.path.join(dirname, 'dataset.npz'))

and study its content

In [12]:
for key in data:
    print(key)

X_train
y_train
X_val
y_val
val_pullbacks


It contains training and validation data and the validation pullbacks. You might ask, what is written in the y_true values for validation and training data, given that we don't know the exact Ricci-flat metric. The 'y_train/val' arrays contain the integration weights and $\Omega \wedge \bar\Omega$ for each point. In principle, they can be used for any relevant pointwise information that could be needed during the training process.

In [13]:
weights = data['y_val'][:, 0]
omega = data['y_val'][:, 1]

we can also compute these values directly with the *PointGenerator*. Note, that the points in 'X_train/val' are floats, because our neural network will work with real values. We can recover the complex points as follows:

In [14]:
points = data['X_val'][:, 0:pg.ncoords] + 1.j*data['X_val'][:, pg.ncoords:]

and then compute the weights

In [15]:
weights2 = pg.point_weight(points)
np.allclose(weights, weights2)

False

and the holomorphic volume form

In [16]:
omega2 = pg.holomorphic_volume_form(points)
omega2 = omega2 * np.conj(omega2)
np.allclose(omega, omega2)

True

We will have to give information of the monomials and their derivatives to the tensorflow model. For this purpose we will create pickled dictionary denoted by *BASIS*.

In [17]:
pg.prepare_basis(dirname, kappa=kappa)

0

Let us look at the information stored in *basis.pickle*

In [18]:
with open(os.path.join(dirname, 'basis.pickle'), 'rb') as f:
    basis = pickle.load(f)
for key in basis:
    print(key)

DQDZB0
DQDZF0
QB0
QF0
NFOLD
AMBIENT
KMODULI
NHYPER
INTNUMS
KAPPA


So in case you want to use your own PointGenerator with our TensorFlow models you will have to create a similar basis dictionary. Here we briefly describe what each of these keys stands for. In general *Q* denotes the defining hypersurface(s) with the final integer digit denoting the hypersurface index. *D* refers to derivatives, *Z* to the ambient space coordinates, *B* to a monomials basis and *F* for factors/coefficients for each monomial.

1. "DQDZB0": $\frac{\partial Q_0}{\partial z_i}$ monomials basis of derivatives of the first (and only for quintic) hypersurface w.r.t. ambient coordinates. 
2. "DQDZF0": $\frac{\partial Q_0}{\partial z_i}$ coefficients of the monomial basis.
3. "QB0": monomials basis for first hypersurface
4. "QF0": coefficients for monomial basis.
5. "NFOLD": CY dimension.
6. "AMBIENT": degrees of projective spaces making up the ambient space.
7. "KMODULI": kähler moduli corresponding to each projective factors. Note the CY needs to be favourable, otherwise you will have some superposition.
8. "NHYPER": number of hypersurfaces.
9. "INTNUMS": The intersection numbers of the CY.
10. "KAPPA": The ratio between the volume measures (Kahler volume over holomorphic volume)

That pretty much sums up our introduction to the *PointGenerator* class, next we will implement the Fermat quintic in the *CICYPointGenerator*.

## CICYPointGenerator

The other point generators come with the same functionality and routines as the *PointGenerator*. 

First we load from the cymetric package.

In [19]:
from cymetric.pointgen.pointgen_cicy import CICYPointGenerator

In contrast to the *PointGenerator* the *CICYPointGenerator* expects a list of monomials and coefficients. We reuse our previous monomials (and coefficients) with

In [20]:
pgcicy = CICYPointGenerator([monomials], [coefficients], kmoduli, ambient)

we again create a dataset

In [21]:
dirname = 'fermat_pgcicy'

In [22]:
kappa = pgcicy.prepare_dataset(n_p, dirname)

/Users/ruehle/venv-ml/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/ruehle/venv-ml/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/Users/ruehle/venv-ml/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/ruehle/venv-ml/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/Users/ruehle/

as you might have realised, this took significantly longer than before. The reason for that is also related to all the warnings. The *CICYPointGenerator* utilises *scipy.optimize.fsolve* to find solutions on CICYs. *fsolve* only provides a single root of the defining hypersurfaces and requires more involved numerics, thus leading to worse performance (also in accuracy).

We again create a basis for the tensorflow models

In [23]:
pgcicy.prepare_basis(dirname, kappa)
with open(os.path.join(dirname, 'basis.pickle'), 'rb') as f:
    basis = pickle.load(f)
for key in basis:
    print(key)

DQDZB0
DQDZF0
QB0
QF0
NFOLD
AMBIENT
KMODULI
NHYPER
INTNUMS
KAPPA


and see that the keys are identical to before.

## PointGeneratorMathematica

As the name suggests, *PointGeneratorMathematica* uses Mathematica as a backend and hence needs mathematica installed on the system. It uses [wolframclient](https://reference.wolfram.com/language/WolframClientForPython/) for interop.

In [24]:
from cymetric.pointgen.pointgen_mathematica import PointGeneratorMathematica

Just like the CICYPointGenerator, *PointGeneratorMathematica* expects a list of monomials and coefficients. We reuse our previous monomials (and coefficients) with

In [25]:
pgmath = PointGeneratorMathematica([monomials], [coefficients], kmoduli, ambient)

we again create a dataset

In [26]:
dirname = 'fermat_pgmath'

In [27]:
kappa = pgmath.prepare_dataset(n_p, dirname)

WolframKernel-<tcp://127.0.0.1:58297>:INFO:Connected to logging socket: tcp://127.0.0.1:58297
WolframKernel-<tcp://127.0.0.1:58297>:DEBUG:Evaluating a new expression.
pointgenMathematica:DEBUG:Running with 10 Mathematica kernels.
WolframKernel-<tcp://127.0.0.1:58297>:INFO:Configuration matrix: {{5}}
WolframKernel-<tcp://127.0.0.1:58297>:INFO:Number of Parameters per P^n: {1}
WolframKernel-<tcp://127.0.0.1:58297>:INFO:Number of points on CY from one ambient space intersection: 5
WolframKernel-<tcp://127.0.0.1:58297>:INFO:Now generating 105263 points...
WolframKernel-<tcp://127.0.0.1:58297>:INFO:Generated 0% of points
WolframKernel-<tcp://127.0.0.1:58297>:INFO:Generated 5% of points
WolframKernel-<tcp://127.0.0.1:58297>:INFO:Generated 10% of points
WolframKernel-<tcp://127.0.0.1:58297>:INFO:Generated 15% of points
WolframKernel-<tcp://127.0.0.1:58297>:INFO:Generated 20% of points
WolframKernel-<tcp://127.0.0.1:58297>:INFO:Generated 25% of points
WolframKernel-<tcp://127.0.0.1:58297>:INFO

We again create a basis for the tensorflow models

In [28]:
pgmath.prepare_basis(dirname, kappa)
with open(os.path.join(dirname, 'basis.pickle'), 'rb') as f:
    basis = pickle.load(f)
for key in basis:
    print(key)

DQDZB0
DQDZF0
QB0
QF0
NFOLD
AMBIENT
KMODULI
NHYPER
INTNUMS
KAPPA


## ToricPointGenerator

The *ToricPointGenerator* is somewhat special and requires additional input data generated from [SageMath](https://www.sagemath.org/). This *toric_data* can be straightforwardly generated in any sage kernel that has access to the *cymetric* package. In praxis only a single module is needed which can be found [here](../cymetric/sage/sagelib.py). 

The next cell won't work in your regular python notebook because it requires some sage routines for toric geometry. See [here](https://doc.sagemath.org/html/en/reference/schemes/sage/schemes/toric/variety.html) for more information about toric varieties and their implementation inn sage and [here](https://doc.sagemath.org/html/en/reference/discrete_geometry/sage/geometry/triangulation/point_configuration.html) for information about triangulations of PointCollections.

We begin by setting up the quintic vertices, which define the fan of the toric ambient variety. After initialising said variety we load the *prepare_toric_cy_data()* routine and generate the neccessary data for the *ToricPointGenerator* and the toric TensorFlow models.

In [4]:
from cymetric.sage.sagelib import prepare_toric_cy_data
import os as os
# Quintic vertices
vertices = [
   [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [-1, -1, -1, -1]
]
origin = [0 for _ in range(len(vertices[0]))]
polytope = LatticePolytope(vertices)
pConfig = PointConfiguration(polytope.points(), star=origin)
#set to topcom for more efficient triangulations
pConfig.set_engine("internal")
# restrict to fine star regular.
triangulations = pConfig.restrict_to_connected_triangulations()
triangulations = triangulations.restrict_to_fine_triangulations()
#triangulations = triangulations.restrict_to_regular_triangulations()
triangulations = triangulations.restrict_to_star_triangulations(origin)
#take first triangulation; build fan and TV
triang = triangulations.triangulate()
tv_fan = triang.fan()
TV = ToricVariety(tv_fan)
fname = os.path.join('fermat_pgtoric', 'toric_data.pickle')
toric_data = prepare_toric_cy_data(TV, fname)

We are now in a position to go back to our regular python kernel or simply continue in Sage and load the *ToricPointGenerator*.

In [5]:
import pickle
from cymetric.pointgen.pointgen_toric import ToricPointGenerator

Let us look at the information that has been written to *toric_data.pickle*.

In [6]:
dirname = 'fermat_pgtoric'
with open(os.path.join(dirname, 'toric_data.pickle'), 'rb') as f:
    toric_data = pickle.load(f)
for key in toric_data:
    print(key)
    print(toric_data[key])

dim_cy
3
vol_j_norm
5
coeff_aK
[(-0.31744931691640943+0.6858155722853458j), (-0.28144566697939244-0.3886386629978263j), (0.8693162553895902-0.37688359896122864j), (-0.2396837356374373+1.879852933717963j), (0.09197351979383012-0.6893621869078683j), (0.03291543678487073-0.8887107461044726j), (0.18135905088276336-0.13468925318125158j), (-0.2623780537957991+1.2478726120019101j), (1.647691886284407-0.0723750727477374j), (1.3505302619239192-0.7589314840633958j), (-0.2592801883048347+1.4941996089310141j), (0.8556500493130682+0.09704127594740121j), (1.0472004799769628-0.8490213656867046j), (-0.24334635356648604-0.08897171185978414j), (0.5189677326464252-0.26569188324102977j), (1.0165655999655385+0.7024945009395085j), (-0.3242203409033504+0.797553986602352j), (-0.762406460658888+0.9357826305438741j), (0.7343603283586707-1.6245786640454434j), (0.004782246967246886+1.1998846111540342j), (0.6035704769872311-1.090620776948206j), (-1.8175045262750547+0.8934656582670933j), (-1.614108876759556+0.63054

We have the following keys:

1. "dim_cy": contains the dimension of the Calabi-Yau.
2. "vol_j_norm": information for normalization of weights.
2. "coeff_aK": are generic complex coefficients in front of each of the defining hypersurface monomials, which you get from the Batyrev construction. Note: Those are by default complex valued as they represent a (redundant) description of the complex moduli.
3. "exp_aK": is the monomial basis for the defining equation.
4. "exps_sections": is a monomial basis for the sections of the kähler cone generators. This one will be needed to generate the integration weights and a Kähler metric in the same Kähler class as our Ricci-flat metric.
5. "patch_masks": are (boolean) coordinate masks for all the patches in the TV.
6. "glsm_charges": are the GLSM charges of the TV.
7. "non_ci_coeffs": when sampling points, we use the projecitivized 0th cohomology class of the Kahler cone generators. There can be non-complete-intersection relations among the sections, which are encoded in the non_ci_coeffs and the non_ci_exps.
8. "non_ci_exps": The exponents of all Kahler cone generator sections
9. "int_nums": are the triple intersection numbers of the TV.

Having loaded the toric_data we can initialize the ToricPointGenerator

In [7]:
pgtoric_gen = ToricPointGenerator(toric_data, kmoduli)

toricpointgen:WARNING:This is outdated code. The integration weight computation            has not been updated yet to match the ToricPointGeneratorMathematica.            In it's current form the integration weights are computed wrongly and            won't give you the correct volume in the MC integration.             Get involved and translate the Mathematica functions to python!            Proceed on your own risk, you have been warned.


We now have a generic quintic with coefficients in front of all 125 monomials.

What if we only want the Fermat Quintic? We can override the information in "coeff_aK" and "exp_aK" with e.g.

In [8]:
toric_data["coeff_aK"] = coefficients
toric_data["exp_aK"] = monomials

and

In [9]:
pgtoric = ToricPointGenerator(toric_data, kmoduli)

toricpointgen:WARNING:This is outdated code. The integration weight computation            has not been updated yet to match the ToricPointGeneratorMathematica.            In it's current form the integration weights are computed wrongly and            won't give you the correct volume in the MC integration.             Get involved and translate the Mathematica functions to python!            Proceed on your own risk, you have been warned.


we continue just as before with creating a dataset

In [13]:
kappa = pgtoric.prepare_dataset(100000, dirname)

pointgen:INFO:Vol_k: 5.0, Vol_cy: 296.38006888745014.


We note, that the data generation again took quite some time even though there should just be a single hypersurface. We also optimize w.r.t. to the sections coming from the Kähler generators and thus have more than a single hypersurface to consider.

We create the *BASIS* containing all information for the TensorFlow models:

In [14]:
pgtoric.prepare_basis(dirname, kappa)
with open(os.path.join(dirname, 'basis.pickle'), 'rb') as f:
    basis = pickle.load(f)
for key in basis:
    print(key)

DQDZB0
DQDZF0
QB0
QF0
NFOLD
AMBIENT
KMODULI
NHYPER
INTNUMS
KAPPA


## ToricPointGeneratorMathematica

The *ToricPointGeneratorMathematica* requires both [SageMath](https://www.sagemath.org/) and Mathematica. SageMath does the toric computations and Mathematica the rest. The *toric_data* can be straightforwardly generated in any sage kernel that has access to the *cymetric* package. In praxis only a single module is needed which can be found [here](../cymetric/sage/sagelib.py). Alternatively, you can run the generator directly from within SageMath, as illustrated in another notebook.

The next cell won't work in your regular python notebook because it requires some sage routines for toric geometry and Mathematica. See [here](https://doc.sagemath.org/html/en/reference/schemes/sage/schemes/toric/variety.html) for more information about toric varieties and their implementation in sage and [here](https://doc.sagemath.org/html/en/reference/discrete_geometry/sage/geometry/triangulation/point_configuration.html) for information about triangulations of PointCollections. For Mathematica interop, it uses the [wolframclient](https://reference.wolfram.com/language/WolframClientForPython/) for interop.

We begin by setting up the quintic vertices, which define the fan of the toric ambient variety. After initialising said variety we load the *prepare_toric_cy_data()* routine and generate the neccessary data for the *ToricPointGenerator* and the toric TensorFlow models.

In [7]:
p = PointConfiguration([[0,0], [0,1], [1,0], [1,1], [-1,-1]]); 
p.restrict_to_connected_triangulations()
p.restrict_to_fine_triangulations()
p.restrict_to_regular_triangulations()


ValueError: You must install TOPCOM to test for regularity.

In [8]:
from cymetric.sage.sagelib import prepare_toric_cy_data
import os as os
# Quintic vertices
vertices = [
   [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [-1, -1, -1, -1]
]
origin = [0 for _ in range(len(vertices[0]))]
polytope = LatticePolytope(vertices)
pConfig = PointConfiguration(polytope.points(), star=origin)
print("made")
#set to topcom for more efficient triangulations
pConfig.set_engine("internal")
print("made")
# restrict to fine star regular.
triangulations = pConfig.restrict_to_connected_triangulations()
triangulations = triangulations.restrict_to_fine_triangulations()
#triangulations = triangulations.restrict_to_regular_triangulations()
triangulations = triangulations.restrict_to_star_triangulations(origin)
#take first triangulation; build fan and TV
triang = triangulations.triangulate()
tv_fan = triang.fan()
TV = ToricVariety(tv_fan)
fname = os.path.join('fermat_pgtoricmath', 'toric_data.pickle')
toric_data = prepare_toric_cy_data(TV, fname)

made
made


We are now in a position to go back to our regular python kernel or simply continue in Sage and load the *PointGeneratorToricMathematica*.

In [38]:
from cymetric.pointgen.pointgen_mathematica import ToricPointGeneratorMathematica

Let us look at the information that has been written to *toric_data.pickle*.

In [11]:
import pickle

In [12]:
dirname = 'fermat_pgtoricmath'
with open(os.path.join(dirname, 'toric_data.pickle'), 'rb') as f:
    toric_data = pickle.load(f)
for key in toric_data:
    print(key)
    print(toric_data[key])

dim_cy
3
vol_j_norm
5
coeff_aK
[(-1.9901231913016204-0.5394472358209468j), (-0.6162604606658445-1.1311105917007802j), (0.5465070356490355+1.0270690861523621j), (0.800524062132159+1.6862662530436507j), (-0.46280729403563486-0.5437361270890179j), (-0.6174798290489638-1.4190378534896697j), (-0.19619697497568406+0.10832419437594047j), (-0.3092467328629806+0.17425761549463384j), (-1.6857303584296655-0.48587464419251714j), (0.10748807422719218-1.2191596060229448j), (-0.2609325815788677+0.29575126277079494j), (0.5140844866932182-0.9413934907898371j), (1.5218365139602716+0.11117215433340737j), (0.3875235041306566+0.4365588444278264j), (-0.9070377602288187-1.294346884962662j), (0.7296701357181227-1.006386272781616j), (2.1468722882046043-0.23302624278543504j), (-0.26583209319507833-0.7418961179720107j), (1.5297987405633624-2.745188583418355j), (1.6069642223255536-0.4682570840334239j), (-1.0515193969163035-1.195436133566121j), (-0.5963367266090399+0.7185328826906052j), (-1.4563602922559098-0.7935

In [39]:
dirname = 'fermat_pgtoricmath'
with open(os.path.join(dirname, 'toric_data.pickle'), 'rb') as f:
    toric_data = pickle.load(f)
for key in toric_data:
    print(key)
    print(toric_data[key])

dim_cy
3
vol_j_norm
5
coeff_aK
[(-0.43795865346198354-1.4542068566020145j), (-0.5203024576289269+2.1810382707504217j), (0.45817739345475533+0.04311395544837721j), (-0.716740668008019+1.6976606816292115j), (1.5039294225696025-1.5158191785465278j), (2.0922840847678352-0.5521020864561111j), (-1.4031308057655576-1.8901548032310274j), (-0.9181269194170236-0.0758327935331521j), (0.5253573750171391+0.21883792874435123j), (0.5173883215219727+2.4216698870875164j), (-1.3787829823445295+0.12499343655331319j), (0.394330611167441-0.17988619298686628j), (-2.252063967252686+0.7643962704166481j), (-0.4608102893199299-0.2595710212508222j), (-0.30183402484417726-0.705006347386717j), (-2.668524488761039-1.4979033181760775j), (-0.21962876151779193-1.6581911705103363j), (-0.09103550588927627+1.3264505309783994j), (0.5060075119224862+2.150661416503481j), (-0.16641488219468464-1.5800538625519214j), (-0.8531180804834545-1.2809549067122166j), (0.5252271458345619+2.745537936682657j), (-0.2838190922800839+1.6431

We have the following keys:

1. "dim_cy": contains the dimension of the Calabi-Yau.
2. "vol_j_norm": information for normalization of weights.
2. "coeff_aK": are generic complex coefficients in front of each of the defining hypersurface monomials, which you get from the Batyrev construction. Note: Those are by default complex valued as they represent a (redundant) description of the complex moduli.
3. "exp_aK": is the monomial basis for the defining equation.
4. "exps_sections": is a monomial basis for the sections of the kähler cone generators. This one will be needed to generate the integration weights and a Kähler metric in the same Kähler class as our Ricci-flat metric.
5. "patch_masks": are (boolean) coordinate masks for all the patches in the TV.
6. "glsm_charges": are the GLSM charges of the TV.
7. "non_ci_coeffs": when sampling points, we use the projecitivized 0th cohomology class of the Kahler cone generators. There can be non-complete-intersection relations among the sections, which are encoded in the non_ci_coeffs and the non_ci_exps.
8. "non_ci_exps": The exponents of all Kahler cone generator sections
9. "int_nums": are the intersection numbers of the TV.

Having loaded the toric_data we can initialize the ToricPointGenerator

In [13]:
pgtoricmath_gen = ToricPointGeneratorMathematica(toric_data, kmoduli)

NameError: name 'ToricPointGeneratorMathematica' is not defined

We now have a generic quintic with coefficients in front of all 125 monomials.

What if we only want the Fermat Quintic? We can override the information in "coeff_aK" and "exp_aK" with e.g.

In [41]:
toric_data["coeff_aK"] = coefficients
toric_data["exp_aK"] = monomials

and

In [42]:
pgtoricmath = ToricPointGeneratorMathematica(toric_data, kmoduli)

we continue just as before with creating a dataset

In [43]:
kappa = pgtoricmath.prepare_dataset(n_p, dirname)

WolframKernel-<tcp://127.0.0.1:58331>:INFO:Connected to logging socket: tcp://127.0.0.1:58331
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Number of points on CY from one ambient space intersection: 5
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Now generating 100000 points...
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Generated 0% of points
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Generated 5% of points
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Generated 10% of points
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Generated 15% of points
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Generated 20% of points
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Generated 25% of points
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Generated 30% of points
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Generated 35% of points
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Generated 40% of points
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Generated 45% of points
WolframKernel-<tcp://127.0.0.1:58331>:INFO:Generated 50% o

We note, that the data generation again took quite some time even though there should just be a single hypersurface. We also optimize w.r.t. to the sections coming from the Kähler generators and thus have more than a single hypersurface to consider.

Also, if you paid close attention to the output, you might have noticed that both Vol_k and Vol_cy are larger by a factor of 5 as compared to the other point generators. This is because the toric model does calculate the correct normalization (i.e. such that vol(X)$=\int J^3=d_{ijk}t^ict^j t^k=\text{mean}(\text{det}(g)/|\Omega|^2 w$) from the triple intersection numbers. To get the same behavior for the other point generators, you need to pass the argument vol_j_norm=5 to the generators and then call prepare_dataset with normalize_to_vol_j=True.

We create the *BASIS* containing all information for the TensorFlow models:

In [44]:
pgtoricmath.prepare_basis(dirname, kappa)
with open(os.path.join(dirname, 'basis.pickle'), 'rb') as f:
    basis = pickle.load(f)
for key in basis:
    print(key)

DQDZB0
DQDZF0
QB0
QF0
NFOLD
AMBIENT
KMODULI
NHYPER
INTNUMS
KAPPA


## Final remarks

In the next notebook we will load the *basis.pickle* and *toric_data.pickle* for the TensorFlow model. The information in *basis.pickle* will override what is written in *toric_data.pickle* so that the models know we want to work with the Fermat and not some generic quintic.

We can also check if the points are all on the fermat:

In [45]:
data = np.load(os.path.join(dirname, 'dataset.npz'))
points = data['X_val'][:,0:pg.ncoords]+1.j*data['X_val'][:,pg.ncoords:]

using the regular *PointGenerator*

In [46]:
np.sum(np.isclose(np.abs(pg.cy_condition(points)), 0))/len(points)

1.0

the CICY

In [47]:
np.sum(np.isclose(np.abs(pgcicy.cy_condition(points)), 0))/len(points)

1.0

the Mathematica

In [48]:
np.sum(np.isclose(np.abs(pgmath.cy_condition(points)), 0))/len(points)

1.0

the toric

In [49]:
np.sum(np.isclose(np.abs(pgtoric.cy_condition(points)), 0))/len(points)

1.0

and the toric mathematica point generators

In [50]:
np.sum(np.isclose(np.abs(pgtoricmath.cy_condition(points)), 0))/len(points)

1.0